# Novelty Algo

## Algorithm to Automate Novelty Scoring

### Import Packages

In [1]:
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
import re
from functools import reduce
import openpyxl
import xlsxwriter

from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.util import ngrams
from nltk import ngrams, FreqDist
from nltk.lm import NgramCounter
import string
import gensim
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors

from spacy.lang.en.stop_words import STOP_WORDS

from collections import Counter
import itertools

from sklearn.metrics.pairwise import cosine_similarity

from scipy import spatial


### Put Data from Excel Sheet into Dataframes

In [2]:
# individual df's for each sheet

# when on pc
data_cup = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Cup")
data_key = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Key")
data_rope = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Rope")
data_brick = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Brick")
data_chair = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Chair")
data_pencil = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Pencil")
data_shoe = pd.read_excel("C:/Users/jhec8/Documents/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Shoe")

# when on mac
# data_cup = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Cup", engine='openpyxl')
# data_key = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Key", engine='openpyxl')
# data_rope = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Rope", engine='openpyxl')
# data_brick = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Brick", engine='openpyxl')
# data_chair = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Chair", engine='openpyxl')
# data_pencil = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Pencil", engine='openpyxl')
# data_shoe = pd.read_excel("/Users/johnhenrycruz/Desktop/Northwestern_SROP/AUT-Scoring/data/autdata_test.xlsx", sheet_name = "Shoe", engine='openpyxl')

### Preprocessing

In [3]:
# nltk corpus stop words
stopwords_nltk = stopwords.words('english')
# spacy stop words
stopwords_spacy = STOP_WORDS

In [35]:
# method to clean the responses
def process_text(text, stopwords_list, stem=True):
    # tokenize text, lemmanize words, removing punctuation, remove stop words, lowercase all words

    # hardcorded for special situations
    text = re.sub("wedging","wedge", text)
    text = re.sub("exersizing","exercising", text)
    text = re.sub("thrown","throw", text)
    
    text = re.sub("/|-"," ", text)
    text = text.translate(str.maketrans('','',string.punctuation))
    tokens = word_tokenize(text)
    
    tokens = [w.lower() for w in tokens]

    tokens = [word for word in tokens if word not in stopwords_list]

    if stem:
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(t) for t in tokens]
#         stemmer = PorterStemmer()
#         tokens = [stemmer.stem(t) for t in tokens]
 
    return tokens

### General Functions

In [5]:
# method to get a list of participants
def get_id_list(df):
    id_list = df['id'].unique()
    id_list = sorted(id_list)
    return id_list

In [6]:
# method to add a new column
# new column are cleaned responses
def get_cleaned_responses(df, stopwords_list):
    # id_df = df[df.id == id]
    df_processed = df.copy(deep=True)
    responses = df['response'].tolist()

    # make list of processed responses
    for response in range(len(responses)):
        responses[response] = process_text(responses[response], stopwords_list, True)

    # add list as column in df
    df_processed['response_processed'] = responses

    return df_processed

## Semantic Spaces
### Word2Vec Models for Embeddings

In [7]:
# load pretrained model
word_model_twitter25 = api.load("glove-twitter-25")

# on pc
# word_model_google = KeyedVectors.load_word2vec_format("C:/Users/jhec8/Documents/Northwestern_SROP/GoogleNews-vectors-negative300.bin", binary=True)

# on mac
# word_model = KeyedVectors.load_word2vec_format("/Users/johnhenrycruz/Desktop/Northwestern_SROP/GoogleNews-vectors-negative300.bin", binary=True)

In [8]:
# create dictionary of counts for each word in model
twitter25_dict = {}
for i in range(len(word_model_twitter25)):
    twitter25_dict[word_model_twitter25.index_to_key[i]] = word_model_twitter25.key_to_index[word_model_twitter25.index_to_key[i]]

In [9]:
# get the frequency of each word in dictionary
total_words = 0
for key in twitter25_dict:
    total_words = total_words + twitter25_dict[key]
    
for key in twitter25_dict:
    twitter25_dict[key] = twitter25_dict[key]/total_words

### ukwac Semantic Space

In [10]:
ukwac_sem_space = pd.read_csv('C:/Users/jhec8/Documents/Northwestern_SROP/ukwac.txt', delimiter = " ", header = None)
ukwac_sem_space = ukwac_sem_space.set_index(0)


In [11]:
ukwac_sem_space

,1,2,3,4,5,6,7,8,9,10,...,391,392,393,394,395,396,397,398,399,400
0,,,,,,,,,,,,,,,,,,,,,
</s>,0.001001,0.001105,-0.000958,-0.000820,0.000342,0.000755,0.000235,0.000053,-0.000901,0.000555,...,-0.000453,0.000009,-0.000691,0.000940,0.000957,0.000815,-0.000368,0.000068,0.000201,-0.000671
the,-0.176528,0.498660,-0.025550,0.454070,0.092507,0.258827,0.260747,0.038257,-0.192332,0.095421,...,0.564220,0.647529,0.264421,0.421305,-0.473167,0.263297,-0.287083,-0.617989,0.086042,0.555010
of,-0.363240,0.207356,-0.333181,0.413527,-0.209578,0.184913,0.446834,0.081978,-0.311497,-0.022150,...,0.804674,0.406484,0.318285,0.755942,-0.478414,0.306891,-0.403508,-0.331963,0.466197,-0.090987
and,-0.146118,0.245265,0.104299,0.577452,0.279372,-0.034110,0.538732,0.020955,-0.177831,0.203355,...,0.624485,0.415114,0.078963,0.336975,-0.113582,0.337823,0.022114,0.170854,-0.132418,-0.645895
be,-0.570585,0.099979,0.554887,-0.479883,0.109848,0.445984,-0.144741,0.347022,-0.093665,0.273852,...,-0.290909,0.302652,-0.019503,0.741136,-0.330540,0.271714,-0.176909,-0.547867,0.180331,0.385517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
kimball,-1.393728,0.242885,-1.556329,0.715039,-0.752086,-0.345446,-0.675950,1.467655,1.450838,-1.472090,...,1.000873,-0.214721,0.226673,1.376971,1.399388,-0.112408,0.651656,-2.597081,0.640036,-1.219004
carara,0.782913,-0.660991,-0.630007,0.463286,0.316139,-0.427366,-0.341664,0.825662,0.116876,-0.553291,...,0.721038,-0.904994,-0.069005,-1.591310,0.341490,0.514781,0.337910,-0.835041,0.882852,-0.214451
hypoallergenic,-0.042689,-0.959848,1.220912,-0.053766,0.180010,-1.810830,-0.816462,-0.248799,1.480133,0.912481,...,0.165383,-0.456424,-0.927048,1.475945,-0.004872,-1.189443,-0.904367,-0.050912,-1.000301,-0.620995


## Novelty Algo 1
### Avg Distance using Word2Vec
### Lesser Average Similarity, More Novel

In [36]:
# method to get the novelty rating
# average of the similarities seen
def get_similarity_word2vec_avg(prompt, phrase_list, word_model):
    avg_sim = 0
    # find similarity of each word in phrase with the prompt
    for term in range(len(phrase_list)):
        avg_sim = avg_sim + word_model.similarity(w1 = prompt, w2 = phrase_list[term])
        
    # take the average
    if len(phrase_list) == 0:
        avg_sim = 0
    else:
        avg_sim = avg_sim/len(phrase_list)

    return (avg_sim)

In [37]:
# method that returns the dataframe with novelty rating for each phrase
def get_novelty_word2vec_avg(df, prompt, stopwords_list, word_model):
    # clean the responses
    novel_rating_df = get_cleaned_responses(df, stopwords_list)
    cleaned_responses = novel_rating_df['response_processed'].tolist()
    avg_sim_list = []

    # implement algo
    # pass in clean responses
    for response in cleaned_responses:
        # add novelty rating to list 
        avg_sim_list.append(get_similarity_word2vec_avg(prompt, response, word_model))

    # add novelty rating list to dataframe
    novel_rating_df['avg_sim'] = avg_sim_list
    
    # new column with novelty rating
    return novel_rating_df

In [38]:
data_brick_novelty_word2vec_avg = get_novelty_word2vec_avg(data_brick, "brick", stopwords_spacy, word_model_twitter25)
data_brick_novelty_word2vec_avg

,id,stim,response,response_processed,avg_sim
0,1476,Brick,as a foundation,[foundation],0.534983
1,1476,Brick,as a weapon,[weapon],0.630485
2,1476,Brick,as a bowl,[bowl],0.665463
3,1718,Brick,build a house,"[build, house]",0.730444
4,1718,Brick,break a window,"[break, window]",0.742416
5,1718,Brick,line a fireplace,"[line, fireplace]",0.682331
6,1718,Brick,line a road,"[line, road]",0.716282
7,1691,Brick,build house,"[build, house]",0.730444
8,1691,Brick,break window,"[break, window]",0.742416
9,1691,Brick,weight for workout,"[weight, workout]",0.557236


## Novelty Algo 2
### Word2Vec + Smooth Inverse Frequency + Cosine Similarity

### Does work well because phrases are super short
### Don't use

In [39]:
# method to apply SIF to the vectors 
def get_sif_feature_vectors(prompt, response, word_model):
    # set the word count dictionary with frequencies
    word_counts = twitter25_dict
    # size of vectore in word embeddings
    embedding_size = 25 
    # set hyper parameter
    a = 0.001
    # list to store vectors
    phrase_set = []
    for phrase in [prompt, response]:
        # zero out the vector
        vs = np.zeros(embedding_size)
        phrase_length = len(phrase)
        for word in phrase:
            # smooth inverse frequency, SIF
            a_value = a / (a + word_counts[word]) 
            # vs += sif * word_vector
            vs = np.add(vs, np.multiply(a_value, word_model[word]))
        # weighted average
        if phrase_length == 0:
            vs[:] = 0
        else:
            vs = np.divide(vs, phrase_length) 
        phrase_set.append(vs)
    # return the SIF adjusted vectors
    return phrase_set

In [40]:
# method to calculate cosine similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

In [41]:
def get_similarity_word2vec_sif_cosinesim(prompt, response, word_model):
    # get the SIF adjusted vectors
    vectors = get_sif_feature_vectors(prompt, response, word_model)
    # return the cosine similarity
    return get_cosine_similarity(vectors[0], vectors[1])

In [42]:
def get_novelty_word2vec_sif_cosinesim(df, prompt, stopwords_list, word_model):
    # clean the responses
    novel_rating_df = get_cleaned_responses(df, stopwords_list)
    cleaned_responses = novel_rating_df['response_processed'].tolist()
    avg_sim_list = []

    # implement algo
    # pass in clean responses
    for response in cleaned_responses:
        # add novelty rating to list 
        avg_sim_list.append(get_similarity_word2vec_sif_cosinesim(prompt, response, word_model))

    # add novelty rating list to dataframe
    novel_rating_df['SIF + cosine sim'] = avg_sim_list
    
    # new column with novelty rating
    return novel_rating_df

In [43]:
data_brick_novelty_word2vec_sif_cosinesim = get_novelty_word2vec_sif_cosinesim(data_brick, "brick", stopwords_spacy, word_model_twitter25)
data_brick_novelty_word2vec_sif_cosinesim

,id,stim,response,response_processed,SIF + cosine sim
0,1476,Brick,as a foundation,[foundation],0.436563
1,1476,Brick,as a weapon,[weapon],0.222613
2,1476,Brick,as a bowl,[bowl],0.484565
3,1718,Brick,build a house,"[build, house]",0.657241
4,1718,Brick,break a window,"[break, window]",0.601265
5,1718,Brick,line a fireplace,"[line, fireplace]",0.496342
6,1718,Brick,line a road,"[line, road]",0.716298
7,1691,Brick,build house,"[build, house]",0.657241
8,1691,Brick,break window,"[break, window]",0.601265
9,1691,Brick,weight for workout,"[weight, workout]",0.417598


## Novelty Algo 3
### ukwac_sem_space + element wise multiplication + cosine distance
### Greater Cos Distance, Greater Novelty

In [44]:
# method to get the element wise multiplied vector
# multiply vectors in phrase
def get_ew_multiplied_vector(phrase_list):
    vectors_list = []
    # add vectors to list
    # change to numpy array
    for term in phrase_list:
        vectors_list.append(np.array(ukwac_sem_space.loc[term].values.tolist()))
    
    # get element wise multiplied vector
    element_wise_multiplied_vector = np.ones(400)

    for vector in vectors_list:
        element_wise_multiplied_vector = element_wise_multiplied_vector * vector

    return element_wise_multiplied_vector

In [45]:
# get cosine sim from prompt and ewm
def get_cosine_sim_ewm(prompt, response):
    prompt_vector = np.array(ukwac_sem_space.loc[prompt].values.tolist())
    ewm_vector = get_ew_multiplied_vector(response)

    return (1 - get_cosine_similarity(prompt_vector, ewm_vector))

In [54]:
# get df with results of the cosine distance from prompt using the elementwise multiplied vectors in the response
def get_novelty_ukwac_multiplication_cosinesim(df, prompt, stopwords_list, model):
    # clean the responses
    novel_rating_df = get_cleaned_responses(df, stopwords_list)
    cleaned_responses = novel_rating_df['response_processed'].tolist()
    # list to store cosine sims
    cosine_sim_list = []

    # implement algo
    # pass in clean responses
    for response in cleaned_responses:
        # add novelty rating to list 
        cosine_sim_list.append(get_cosine_sim_ewm(prompt, response))

    # add novelty rating list to dataframe
    novel_rating_df['ewm_vector + cosine_dis'] = cosine_sim_list
    
    # new column with novelty rating
    return novel_rating_df

In [55]:
test1 = np.array(ukwac_sem_space.loc['house'].values.tolist())
test2 = np.array(ukwac_sem_space.loc['build'].values.tolist())
test3 = test1 * test2
test4 = np.array(ukwac_sem_space.loc['brick'].values.tolist())
print(cosine_similarity(test4.reshape(1, -1), test3.reshape(1, -1))[0][0])

0.04491584989139645


In [56]:
data_brick_novelty_ukwac_ewm_cosinesim = get_novelty_ukwac_multiplication_cosinesim(data_brick, 'brick', stopwords_spacy, word_model_twitter25)
data_brick_novelty_ukwac_ewm_cosinesim

,id,stim,response,response_processed,ewm_vector + cosine_dis
0,1476,Brick,as a foundation,[foundation],0.936781
1,1476,Brick,as a weapon,[weapon],0.956177
2,1476,Brick,as a bowl,[bowl],0.779148
3,1718,Brick,build a house,"[build, house]",0.955084
4,1718,Brick,break a window,"[break, window]",0.978611
5,1718,Brick,line a fireplace,"[line, fireplace]",1.043541
6,1718,Brick,line a road,"[line, road]",0.976273
7,1691,Brick,build house,"[build, house]",0.955084
8,1691,Brick,break window,"[break, window]",0.978611
9,1691,Brick,weight for workout,"[weight, workout]",1.033559


## Novelty Algo 4
### ukwac_sem_space + local minina + cosine distance

In [52]:
# get word in phrase that has the least distance from the prompt
def get_minima_vector_cos_distance(prompt, phrase_list):
    distances_list = []
    prompt_vector = np.array(ukwac_sem_space.loc[prompt].values.tolist())
    
    for term in phrase_list:
        distances_list.append((1 - get_cosine_similarity(prompt_vector, np.array(ukwac_sem_space.loc[term].values.tolist()))))
        
    return max(distances_list)

In [53]:
# get df with results of the cosine distance from prompt using the minima vector in the response
def get_novelty_ukwac_minimavec_cosinedist(df, prompt, stopwords_list, model):
    # clean the responses
    novel_rating_df = get_cleaned_responses(df, stopwords_list)
    cleaned_responses = novel_rating_df['response_processed'].tolist()
    # list to store cosine sims
    cosine_sim_list = []

    # implement algo
    # pass in clean responses
    for response in cleaned_responses:
        # add novelty rating to list 
        cosine_sim_list.append(get_minima_vector_cos_distance(prompt, response))

    # add novelty rating list to dataframe
    novel_rating_df['minima_vector + cosine_sim'] = cosine_sim_list
    
    # new column with novelty rating
    return novel_rating_df

In [51]:
data_brick_novelty_ukwac_minimavec_cosinedist = get_novelty_ukwac_minimavec_cosinedist(data_brick, 'brick', stopwords_spacy, word_model_twitter25)
data_brick_novelty_ukwac_minimavec_cosinedist

,id,stim,response,response_processed,minima_vector + cosine_sim
0,1476,Brick,as a foundation,[foundation],0.936781
1,1476,Brick,as a weapon,[weapon],0.956177
2,1476,Brick,as a bowl,[bowl],0.779148
3,1718,Brick,build a house,"[build, house]",0.609464
4,1718,Brick,break a window,"[break, window]",0.905654
5,1718,Brick,line a fireplace,"[line, fireplace]",0.801291
6,1718,Brick,line a road,"[line, road]",0.801291
7,1691,Brick,build house,"[build, house]",0.609464
8,1691,Brick,break window,"[break, window]",0.905654
9,1691,Brick,weight for workout,"[weight, workout]",1.022161


### Write Results into Excel Sheet

In [30]:
prompts_list = ['cup', 'key', 'rope', 'brick', 'chair', 'pencil', 'shoe']
data_list = [data_cup, data_key, data_rope, data_brick, data_chair, data_pencil, data_shoe]

In [31]:
# write results df of each dataset for specific methods
def write_results_excel(method, stopwords, model):
    # change this when writing new sheet
    writer = pd.ExcelWriter('novelty_algo_3_results.xlsx', engine='xlsxwriter')

    for i in range(len(prompts_list)):
        df = method(data_list[i], prompts_list[i], stopwords, model)
        df.to_excel(writer, sheet_name = prompts_list[i], index = False)
    writer.save()

In [32]:
# write_results_excel(get_novelty_word2vec_avg, stopwords_spacy, word_model_twitter25)

In [33]:
# write_results_excel(get_novelty_word2vec_sif_cosinesim, stopwords_spacy, word_model_twitter25)

In [34]:
# write_results_excel(get_novelty_ukwac_multiplication_cosinesim, stopwords_spacy, word_model_twitter25)

Algo Design Brainstorming:
* Word2vec, average similarity between words in two responses
* Problems:
    * phrases vs words
    * compare between just two responses or the whole list of responses
* Algo Idea
    * find similarity between prompt and average of word in response
    * the average is the value that represents novelty
* Things to Look Into:
    * doc2Vec - look into sentence vector
        * follow similar logic to above but no need to average for multiple words
    * Word2Vec + SIF + Cosine Similarity
    * Word2Vec + WMD 
* Course of Actions for Missing Words/Misspellings?
    * hand remove?
* What to do when whole phrase stop words
    * make 0

* To Do List
- [x] write preprocessing methods
- [x] write out initial algo
- [x] implement first algo idea
- [x] set up work environment on macbook
- [x] look into Word2Vec + SIF + Cosine Similarity
- [x] set up excel sheet download with different sheets
- [ ] look into doc2vec 
- [ ] look into Word2Vec + WMD 
- [ ] look at https://github.com/PrincetonML/SIF for better SIF 
